In [1]:
# pyright: reportMissingImports=false, reportUnusedVariable=warning, reportUntypedBaseClass=error
from database import Database
from fingerprinting import fingerprints
from peaks import find_neighborhood, find_min_amp

import numpy as np
import librosa
# import matplotlib.pyplot as plt
from IPython.display import Audio
from typing import Union, Callable, Tuple, List
from pathlib import Path

import matplotlib.mlab as mlab

from scipy.ndimage.filters import maximum_filter
from scipy.ndimage.morphology import generate_binary_structure, binary_erosion
from scipy.ndimage.morphology import iterate_structure

# %matplotlib notebook

database = Database()

In [5]:
from numba import njit

# `@njit` "decorates" the `_peaks` function. This tells Numba to
# compile this function using the "low level virtual machine" (LLVM)
# compiler. The resulting object is a Python function that, when called,
# executes optimized machine code instead of the Python code
# 
# The code used in _peaks adheres strictly to the subset of Python and
# NumPy that is supported by Numba's jit. This is a requirement in order
# for Numba to know how to compile this function to more efficient
# instructions for the machine to execute
@njit
def _peaks(
    data_2d: np.ndarray, rows: np.ndarray, cols: np.ndarray, amp_min: float
) -> List[Tuple[int, int]]:
    """
    A Numba-optimized 2-D peak-finding algorithm.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected.

    rows : numpy.ndarray, shape-(N,)
        The 0-centered row indices of the local neighborhood mask
    
    cols : numpy.ndarray, shape-(N,)
        The 0-centered column indices of the local neighborhood mask
        
    amp_min : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location. 
    """
    peaks = []  # stores the (row, col) locations of all the local peaks

    # Iterate over the 2-D data in col-major order
    # we want to see if there is a local peak located at
    # row=r, col=c
    for c, r in np.ndindex(*data_2d.shape[::-1]):
        if data_2d[r, c] <= amp_min:
            # The amplitude falls beneath the minimum threshold
            # thus this can't be a peak.
            continue
        
        # Iterating over the neighborhood centered on (r, c)
        # dr: displacement from r
        # dc: discplacement from c
        for dr, dc in zip(rows, cols):
            if dr == 0 and dc == 0:
                # This would compare (r, c) with itself.. skip!
                continue

            if not (0 <= r + dr < data_2d.shape[0]):
                # neighbor falls outside of boundary
                continue

            # mirror over array boundary
            if not (0 <= c + dc < data_2d.shape[1]):
                # neighbor falls outside of boundary
                continue

            if data_2d[r, c] < data_2d[r + dr, c + dc]:
                # One of the amplitudes within the neighborhood
                # is larger, thus data_2d[r, c] cannot be a peak
                break
        else:
            # if we did not break from the for-loop then (r, c) is a peak
            peaks.append((r, c))
    return peaks

def local_peak_locations(data_2d: np.ndarray, neighborhood: np.ndarray, amp_min: float):
    """
    Defines a local neighborhood and finds the local peaks
    in the spectrogram, which must be larger than the specified `amp_min`.
    
    Parameters
    ----------
    data_2d : numpy.ndarray, shape-(H, W)
        The 2D array of data in which local peaks will be detected
    
    neighborhood : numpy.ndarray, shape-(h, w)
        A boolean mask indicating the "neighborhood" in which each
        datum will be assessed to determine whether or not it is
        a local peak. h and w must be odd-valued numbers
        
    amp_min : float
        All amplitudes at and below this value are excluded from being local 
        peaks.
    
    Returns
    -------
    List[Tuple[int, int]]
        (row, col) index pair for each local peak location.
    
    Notes
    -----
    Neighborhoods that overlap with the boundary are mirrored across the boundary.
    
    The local peaks are returned in column-major order.
    """
    rows, cols = np.where(neighborhood)
    assert neighborhood.shape[0] % 2 == 1
    assert neighborhood.shape[1] % 2 == 1

    # center neighborhood indices around center of neighborhood
    rows -= neighborhood.shape[0] // 2
    cols -= neighborhood.shape[1] // 2

    return _peaks(data_2d, rows, cols, amp_min=amp_min)


In [11]:
def generate_fingerprints(samples, sampling_rate):
    # perform rfft
    N = len(samples)
    T = N / sampling_rate

    times = np.arange(N) / sampling_rate
    c_k = np.fft.rfft(samples) 
      # convert ck (complex Fourier coeff) -> |ak| (real-valued coeff)
    amps = np.abs(c_k) / N
    amps[1 : (-1 if N % 2 == 0 else None)] *= 2

      # convert k = 0, 1, ... to freq = 0, 1/T, 2/T, ..., (int(N/2) + 1)/T
    freq = np.arange(len(amps)) / T

    spectrogram, freqs, times = mlab.specgram(
        samples,
        NFFT=4096,
        Fs=sampling_rate,
        window=mlab.window_hanning,
        noverlap=int(4096 / 2)
    )


    neighborhood = find_neighborhood(2, 1, 20)
    min_amp = find_min_amp(spectrogram)

    peaks = local_peak_locations(spectrogram, neighborhood, min_amp)
    song_fingerprints, times = fingerprints(15, peaks)
    
    return (song_fingerprints, times)

In [12]:
def import_song(path, song_name):
  # sample the audio file
  samples, sampling_rate = librosa.load(path, sr=44100, mono=True)

  song_fingerprints, times = generate_fingerprints(samples, sampling_rate)

  database.store_fingerprints(song_fingerprints, song_name, times)


In [9]:
import os
song_directory = r"C:\Users\josep\Documents\GitHub\cogzam\music"

for filename in os.listdir(song_directory):
    print("Importing: ", filename)
    file_path = song_directory + "\\" + filename
    import_song(file_path, filename)

Importing:  anti-romantic.mp3
Importing:  best-of-me.mp3
Importing:  cemetary.mp3
Importing:  drivers-license.mp3
Importing:  good4u.mp3
Importing:  jealousy.mp3
Importing:  lifegoeson.mp3
Importing:  pleasedontgo.mp3
Importing:  popstars.mp3
Importing:  psycho.mp3
Importing:  someoneyouloved.mp3
Importing:  speechless.mp3


In [19]:
import pickle

# load "microphone" samples
clips_directory = r"C:\Users\josep\Documents\GitHub\cogzam\cogzam\clips\\"
clips_name = r"drivers_license_clips.pkl"

with open(clips_directory + clips_name, mode="rb") as clips_file:
    clips = pickle.load(clips_file)
    
    for sample in clips:
        sample_fingerprints, times = generate_fingerprints(sample, 44100)
        
        counts = database.search_song(sample_fingerprints, times)
        
        print(counts[1])
        
        
            

[(31, 63, 0), (31, 94, 0), (31, 691, 0), (31, 182, 4), (31, 209, 4), (31, 364, 4), (31, 258, 5), (31, 284, 5), (31, 230, 6), (31, 563, 6), (31, 1029, 12), (31, 408, 13), (31, 622, 13), (31, 323, 14), (31, 666, 14)]
[(63, 94, 0), (63, 691, 0), (63, 182, 4), (63, 209, 4), (63, 364, 4), (63, 258, 5), (63, 284, 5), (63, 230, 6), (63, 563, 6), (63, 1029, 12), (63, 408, 13), (63, 622, 13), (63, 323, 14), (63, 666, 14), (63, 47, 22)]
[(94, 691, 0), (94, 182, 4), (94, 209, 4), (94, 364, 4), (94, 258, 5), (94, 284, 5), (94, 230, 6), (94, 563, 6), (94, 1029, 12), (94, 408, 13), (94, 622, 13), (94, 323, 14), (94, 666, 14), (94, 47, 22), (94, 94, 22)]
[(691, 182, 4), (691, 209, 4), (691, 364, 4), (691, 258, 5), (691, 284, 5), (691, 230, 6), (691, 563, 6), (691, 1029, 12), (691, 408, 13), (691, 622, 13), (691, 323, 14), (691, 666, 14), (691, 47, 22), (691, 94, 22), (691, 117, 22)]
[(182, 209, 0), (182, 364, 0), (182, 258, 1), (182, 284, 1), (182, 230, 2), (182, 563, 2), (182, 1029, 8), (182, 408, 9

[(689, 1017, 1), (689, 238, 9), (689, 383, 9), (689, 199, 11), (689, 259, 11), (689, 857, 15), (689, 893, 15), (689, 918, 15), (689, 955, 15), (689, 1007, 15), (689, 1059, 15), (689, 1097, 15), (689, 405, 16), (689, 505, 16), (689, 576, 16)]
[(1017, 238, 8), (1017, 383, 8), (1017, 199, 10), (1017, 259, 10), (1017, 857, 14), (1017, 893, 14), (1017, 918, 14), (1017, 955, 14), (1017, 1007, 14), (1017, 1059, 14), (1017, 1097, 14), (1017, 405, 15), (1017, 505, 15), (1017, 576, 15), (1017, 675, 15)]
[(238, 383, 0), (238, 199, 2), (238, 259, 2), (238, 857, 6), (238, 893, 6), (238, 918, 6), (238, 955, 6), (238, 1007, 6), (238, 1059, 6), (238, 1097, 6), (238, 405, 7), (238, 505, 7), (238, 576, 7), (238, 675, 7), (238, 439, 10)]
[(383, 199, 2), (383, 259, 2), (383, 857, 6), (383, 893, 6), (383, 918, 6), (383, 955, 6), (383, 1007, 6), (383, 1059, 6), (383, 1097, 6), (383, 405, 7), (383, 505, 7), (383, 576, 7), (383, 675, 7), (383, 439, 10), (383, 162, 13)]
[(199, 259, 0), (199, 857, 4), (199, 893

[(102, 1730, 0), (102, 21, 1), (102, 1858, 2), (102, 43, 5), (102, 185, 5), (102, 758, 5), (102, 801, 5), (102, 1301, 5), (102, 1330, 5), (102, 1356, 5), (102, 1714, 5), (102, 1776, 5), (102, 2016, 5), (102, 2043, 5), (102, 1905, 6)]
[(1730, 21, 1), (1730, 1858, 2), (1730, 43, 5), (1730, 185, 5), (1730, 758, 5), (1730, 801, 5), (1730, 1301, 5), (1730, 1330, 5), (1730, 1356, 5), (1730, 1714, 5), (1730, 1776, 5), (1730, 2016, 5), (1730, 2043, 5), (1730, 1905, 6), (1730, 1842, 7)]
[(21, 1858, 1), (21, 43, 4), (21, 185, 4), (21, 758, 4), (21, 801, 4), (21, 1301, 4), (21, 1330, 4), (21, 1356, 4), (21, 1714, 4), (21, 1776, 4), (21, 2016, 4), (21, 2043, 4), (21, 1905, 5), (21, 1842, 6), (21, 1086, 9)]
[(1858, 43, 3), (1858, 185, 3), (1858, 758, 3), (1858, 801, 3), (1858, 1301, 3), (1858, 1330, 3), (1858, 1356, 3), (1858, 1714, 3), (1858, 1776, 3), (1858, 2016, 3), (1858, 2043, 3), (1858, 1905, 4), (1858, 1842, 5), (1858, 1086, 8), (1858, 114, 9)]
[(43, 185, 0), (43, 758, 0), (43, 801, 0), (43

[(433, 579, 0), (433, 1016, 0), (433, 1054, 0), (433, 249, 1), (433, 274, 1), (433, 340, 1), (433, 363, 1), (433, 454, 1), (433, 477, 1), (433, 507, 3), (433, 543, 3), (433, 736, 3), (433, 893, 3), (433, 115, 4), (433, 193, 4)]
[(579, 1016, 0), (579, 1054, 0), (579, 249, 1), (579, 274, 1), (579, 340, 1), (579, 363, 1), (579, 454, 1), (579, 477, 1), (579, 507, 3), (579, 543, 3), (579, 736, 3), (579, 893, 3), (579, 115, 4), (579, 193, 4), (579, 869, 4)]
[(1016, 1054, 0), (1016, 249, 1), (1016, 274, 1), (1016, 340, 1), (1016, 363, 1), (1016, 454, 1), (1016, 477, 1), (1016, 507, 3), (1016, 543, 3), (1016, 736, 3), (1016, 893, 3), (1016, 115, 4), (1016, 193, 4), (1016, 869, 4), (1016, 983, 4)]
[(1054, 249, 1), (1054, 274, 1), (1054, 340, 1), (1054, 363, 1), (1054, 454, 1), (1054, 477, 1), (1054, 507, 3), (1054, 543, 3), (1054, 736, 3), (1054, 893, 3), (1054, 115, 4), (1054, 193, 4), (1054, 869, 4), (1054, 983, 4), (1054, 21, 7)]
[(249, 274, 0), (249, 340, 0), (249, 363, 0), (249, 454, 0), (

[(61, 274, 0), (61, 730, 0), (61, 791, 0), (61, 122, 1), (61, 365, 1), (61, 426, 1), (61, 638, 1), (61, 699, 1), (61, 309, 2), (61, 96, 4), (61, 194, 4), (61, 849, 4), (61, 886, 4), (61, 928, 4), (61, 1142, 4)]
[(274, 730, 0), (274, 791, 0), (274, 122, 1), (274, 365, 1), (274, 426, 1), (274, 638, 1), (274, 699, 1), (274, 309, 2), (274, 96, 4), (274, 194, 4), (274, 849, 4), (274, 886, 4), (274, 928, 4), (274, 1142, 4), (274, 11, 5)]
[(730, 791, 0), (730, 122, 1), (730, 365, 1), (730, 426, 1), (730, 638, 1), (730, 699, 1), (730, 309, 2), (730, 96, 4), (730, 194, 4), (730, 849, 4), (730, 886, 4), (730, 928, 4), (730, 1142, 4), (730, 11, 5), (730, 465, 5)]
[(791, 122, 1), (791, 365, 1), (791, 426, 1), (791, 638, 1), (791, 699, 1), (791, 309, 2), (791, 96, 4), (791, 194, 4), (791, 849, 4), (791, 886, 4), (791, 928, 4), (791, 1142, 4), (791, 11, 5), (791, 465, 5), (791, 499, 5)]
[(122, 365, 0), (122, 426, 0), (122, 638, 0), (122, 699, 0), (122, 309, 1), (122, 96, 3), (122, 194, 3), (122, 849

[(463, 590, 0), (463, 793, 0), (463, 241, 3), (463, 510, 3), (463, 679, 3), (463, 164, 5), (463, 625, 5), (463, 649, 5), (463, 729, 5), (463, 769, 5), (463, 819, 5), (463, 849, 5), (463, 881, 5), (463, 912, 5), (463, 1058, 5)]
[(590, 793, 0), (590, 241, 3), (590, 510, 3), (590, 679, 3), (590, 164, 5), (590, 625, 5), (590, 649, 5), (590, 729, 5), (590, 769, 5), (590, 819, 5), (590, 849, 5), (590, 881, 5), (590, 912, 5), (590, 1058, 5), (590, 1080, 5)]
[(793, 241, 3), (793, 510, 3), (793, 679, 3), (793, 164, 5), (793, 625, 5), (793, 649, 5), (793, 729, 5), (793, 769, 5), (793, 819, 5), (793, 849, 5), (793, 881, 5), (793, 912, 5), (793, 1058, 5), (793, 1080, 5), (793, 1120, 5)]
[(241, 510, 0), (241, 679, 0), (241, 164, 2), (241, 625, 2), (241, 649, 2), (241, 729, 2), (241, 769, 2), (241, 819, 2), (241, 849, 2), (241, 881, 2), (241, 912, 2), (241, 1058, 2), (241, 1080, 2), (241, 1120, 2), (241, 1159, 2)]
[(510, 679, 0), (510, 164, 2), (510, 625, 2), (510, 649, 2), (510, 729, 2), (510, 769,

[(43, 126, 4), (43, 181, 4), (43, 75, 5), (43, 526, 10), (43, 574, 10), (43, 622, 10), (43, 774, 10), (43, 895, 10), (43, 1038, 10), (43, 1104, 10), (43, 932, 11), (43, 1164, 11), (43, 231, 15), (43, 289, 15), (43, 358, 15)]
[(126, 181, 0), (126, 75, 1), (126, 526, 6), (126, 574, 6), (126, 622, 6), (126, 774, 6), (126, 895, 6), (126, 1038, 6), (126, 1104, 6), (126, 932, 7), (126, 1164, 7), (126, 231, 11), (126, 289, 11), (126, 358, 11), (126, 404, 11)]
[(181, 75, 1), (181, 526, 6), (181, 574, 6), (181, 622, 6), (181, 774, 6), (181, 895, 6), (181, 1038, 6), (181, 1104, 6), (181, 932, 7), (181, 1164, 7), (181, 231, 11), (181, 289, 11), (181, 358, 11), (181, 404, 11), (181, 465, 11)]
[(75, 526, 5), (75, 574, 5), (75, 622, 5), (75, 774, 5), (75, 895, 5), (75, 1038, 5), (75, 1104, 5), (75, 932, 6), (75, 1164, 6), (75, 231, 10), (75, 289, 10), (75, 358, 10), (75, 404, 10), (75, 465, 10), (75, 506, 10)]
[(526, 574, 0), (526, 622, 0), (526, 774, 0), (526, 895, 0), (526, 1038, 0), (526, 1104, 0

[(409, 702, 1), (409, 825, 1), (409, 11, 2), (409, 583, 2), (409, 613, 2), (409, 933, 2), (409, 984, 2), (409, 1061, 2), (409, 1087, 2), (409, 1172, 2), (409, 163, 3), (409, 368, 3), (409, 449, 3), (409, 82, 4), (409, 122, 4)]
[(702, 825, 0), (702, 11, 1), (702, 583, 1), (702, 613, 1), (702, 933, 1), (702, 984, 1), (702, 1061, 1), (702, 1087, 1), (702, 1172, 1), (702, 163, 2), (702, 368, 2), (702, 449, 2), (702, 82, 3), (702, 122, 3), (702, 898, 6)]
[(825, 11, 1), (825, 583, 1), (825, 613, 1), (825, 933, 1), (825, 984, 1), (825, 1061, 1), (825, 1087, 1), (825, 1172, 1), (825, 163, 2), (825, 368, 2), (825, 449, 2), (825, 82, 3), (825, 122, 3), (825, 898, 6), (825, 287, 9)]
[(11, 583, 0), (11, 613, 0), (11, 933, 0), (11, 984, 0), (11, 1061, 0), (11, 1087, 0), (11, 1172, 0), (11, 163, 1), (11, 368, 1), (11, 449, 1), (11, 82, 2), (11, 122, 2), (11, 898, 5), (11, 287, 8), (11, 205, 9)]
[(583, 613, 0), (583, 933, 0), (583, 984, 0), (583, 1061, 0), (583, 1087, 0), (583, 1172, 0), (583, 163, 1

[(93, 139, 0), (93, 373, 0), (93, 420, 0), (93, 654, 0), (93, 184, 1), (93, 276, 2), (93, 308, 2), (93, 396, 2), (93, 115, 7), (93, 463, 7), (93, 532, 7), (93, 593, 7), (93, 61, 10), (93, 935, 11), (93, 989, 11)]
[(139, 373, 0), (139, 420, 0), (139, 654, 0), (139, 184, 1), (139, 276, 2), (139, 308, 2), (139, 396, 2), (139, 115, 7), (139, 463, 7), (139, 532, 7), (139, 593, 7), (139, 61, 10), (139, 935, 11), (139, 989, 11), (139, 1024, 11)]
[(373, 420, 0), (373, 654, 0), (373, 184, 1), (373, 276, 2), (373, 308, 2), (373, 396, 2), (373, 115, 7), (373, 463, 7), (373, 532, 7), (373, 593, 7), (373, 61, 10), (373, 935, 11), (373, 989, 11), (373, 1024, 11), (373, 1055, 11)]
[(420, 654, 0), (420, 184, 1), (420, 276, 2), (420, 308, 2), (420, 396, 2), (420, 115, 7), (420, 463, 7), (420, 532, 7), (420, 593, 7), (420, 61, 10), (420, 935, 11), (420, 989, 11), (420, 1024, 11), (420, 1055, 11), (420, 1086, 11)]
[(654, 184, 1), (654, 276, 2), (654, 308, 2), (654, 396, 2), (654, 115, 7), (654, 463, 7), 

[(1080, 1110, 0), (1080, 1140, 0), (1080, 183, 2), (1080, 275, 2), (1080, 229, 3), (1080, 367, 3), (1080, 413, 3), (1080, 689, 3), (1080, 736, 3), (1080, 828, 3), (1080, 782, 4), (1080, 862, 4), (1080, 1055, 4), (1080, 321, 5), (1080, 629, 8)]
[(1110, 1140, 0), (1110, 183, 2), (1110, 275, 2), (1110, 229, 3), (1110, 367, 3), (1110, 413, 3), (1110, 689, 3), (1110, 736, 3), (1110, 828, 3), (1110, 782, 4), (1110, 862, 4), (1110, 1055, 4), (1110, 321, 5), (1110, 629, 8), (1110, 439, 9)]
[(1140, 183, 2), (1140, 275, 2), (1140, 229, 3), (1140, 367, 3), (1140, 413, 3), (1140, 689, 3), (1140, 736, 3), (1140, 828, 3), (1140, 782, 4), (1140, 862, 4), (1140, 1055, 4), (1140, 321, 5), (1140, 629, 8), (1140, 439, 9), (1140, 541, 9)]
[(183, 275, 0), (183, 229, 1), (183, 367, 1), (183, 413, 1), (183, 689, 1), (183, 736, 1), (183, 828, 1), (183, 782, 2), (183, 862, 2), (183, 1055, 2), (183, 321, 3), (183, 629, 6), (183, 439, 7), (183, 541, 7), (183, 576, 7)]
[(275, 229, 1), (275, 367, 1), (275, 413, 1)

In [10]:
database.save_database()

# Test loading from database instead of importing